In [ ]:
!pip install beautifulsoup
!pip install selenium

In [ ]:
#install webdriver
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
# importing libraries
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import requests

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

!pip install dateparser
import dateparser

In [ ]:
def get_driver():

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage') 
    chrome_options.add_argument("--window-size=1920,1080")

    
    driver = webdriver.Chrome('chromedriver',options=chrome_options)

    return driver

In [ ]:
def get_url(n):
    """Function used for indexing pages given this url as first url 
    https://www.indeed.com/jobs?l=Rancho%20Cordova%2C%20CA&start=0&vjk=44d029cff87a579f
    """
    urls = []
 
    for i in range(n):
        urls.append(f'https://www.indeed.com/jobs?l=Rancho%20Cordova%2C%20CA&start={i*10}&vjk=44d029cff87a579f')
    return urls

In [ ]:
def get_job_info(card, soup):
 
    title = soup.find("h2", class_="jobTitle").get_text()
    
    company = card.find('span','companyName').text.strip()
    
    location = card.find('div','companyLocation').text
    
    summary = card.find('div','job-snippet').text.strip()
    
    post_date = card.find('span','date').text
    date = dateparser.parse(post_date).strftime("%Y-%m-%d")
    

    link = 'https://www.indeed.com' + card.get('href')
    

    driver.get(link)
    main = WebDriverWait(driver,30).until(
            EC.presence_of_element_located((By.XPATH,'//*[@id="viewJobSSRRoot"]/div/div[3]/div/div/div[1]'))
        )
   
    job_desc = main.find_element_by_xpath('//*[@id="jobDescriptionText"]').text 
    print(job_desc)
    try:
        salary = card.find('span','salaryText').text
    except AttributeError:
        salary = ''

    record = {'title': title, 
        'company':company,
        'location': location,
        'post_date': date,
        'salary': salary,
        'summary': summary,
        'link': link,
        'description':job_desc}

    #record = (title, company, location, post_date,salary,  summary, link, job_desc)

    return record

In [ ]:
driver = get_driver()
urls = get_url(25)
print(urls)
headers = {'User Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}
joblist = []

for url in urls:
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    cards = soup.find_all('a', 'tapItem')
    
    for card in cards:
        try:
            record = get_job_info(card, soup)
            print('ok')
            joblist.append(record)
        except:
          pass
          
print(joblist)


In [ ]:
len(joblist)

In [ ]:
df = pd.DataFrame(joblist)

df.head()

In [ ]:
df.to_csv('indeed_jobs.csv', index = False)

# New Section